# CellProfiling  in Python

A high-throughput screening analysis pipeline, similar to what you would do in CellProfiler, implemented in Python.

Aims: 

* create an image analysis pipeline for batch processing, somewhat similar to running Cellprofiler
* do some statistical analysis and create interactive visualizations using holoviews
* introduce pandas data frames

# High Throughput Screening Workflow

<img src="./Illustrations/HTMPipeline_alt.png" height=800>

# Sample images
Images are a subset of dataset BBBC022 from the [Broad Bioimage Benchmark Collection](https://data.broadinstitute.org/bbbc/)

The following description of the dataset is from their website (https://data.broadinstitute.org/bbbc/BBBC022/):

Description of the biological application
Phenotypic profiling attempts to summarize multiparametric, feature-based analysis of cellular phenotypes of each sample so that similarities between profiles reflect similarities between samples. This image set provides a basis for testing image-based profiling methods wrt. to their ability to distinguish the effects of small molecules.

Images
The images are of U2OS cells treated with each of 1600 known bioactive compounds and labeled with six labels that characterize seven organelles (the “Cell Painting” assay).

This pilot experiment consists of 20 plates. Each plate has 384 wells and each well has 9 fields of view for a total of 69,120 fields of view. Each field was imaged in five channels (detection wavelengths), and each channel is stored as a separate, grayscale image file, so there are 345,600 image files in 16-bit TIFF format.

The images are provided in 100 zip archive files, one for each combination of plate and channel. There is a list of the URLs of all 100 files to facilitate downloading the entire image set in batch.

In [ ]:
# Import modules
import pathlib
import re
import numpy as np
import mahotas # see other Image Analysis Packages !
import scipy.ndimage.morphology
import pandas as pd
from skimage.io import imread

In [ ]:
#files = glob.glob("/Users/volker/Downloads/BroadData/**/*.tif", recursive=True)

## Dealing with filenames in python, the 2018 way 

Consider using `pathlib` instead of `os.path`.


In [ ]:
# Set our base folder (adjust this to the path where your images are)
folder = pathlib.Path("/Users/volker/Downloads/BroadData/for_course/")

In [ ]:
# rglob "recursive" (i.e. look in subfolders) "glob" (search for files) according to a pattern (here "*.tif")
files =  folder.rglob("*.tif")
    

In [ ]:
# But what is this thing ? Doesn't look like a list of filenames !
files

It's a _generator_ object ... you will find these get returned quite often in Python 3 (also in places where you would have gotten back a list in python 2.7). 


You can convert the generator object into a list using `list()`. 

In [ ]:
allfiles = list(files)
allfiles

Now try to run the above cell again ... What's happening?

The generator generates each item only once !

So what is the benefit ?

**only compute when needed** 

In [ ]:
#%%timeit
files =  folder.rglob("*.tif")
import itertools
first_5_files = itertools.islice(files, 5)
first_5_files = list(first_5_files)

In [ ]:
#%%timeit 
files =  folder.rglob("*.tif")
first_5_files = list(files)[:5]
first_5_files

# Extracting Metadata from Path/File names with regular expressions

In many cases, you will find some Metadata that is related to your screen embedded in the file names of the images.
Therefore we need to extract this information from the images we analyze.

_Regular expressions_ are a flexible tool for analyzing/splitting strings that are built according to some regular pattern. If you google for "python regular expressions" you will find plenty of documentation and examples on how to use them. 

As regular expressions have a large number of building blocks that may be difficult to remember, it is nice to have a cheat sheet. There is a great online tool for creating and debugging regular expressions with a built-in cheat sheet, namely (http://regex101.com). Make sure you select Python on the left.

<img src="./Illustrations/regex101.png" width=800>

In [ ]:
# this regular expression should work, give it a try in regex101 
# you can also try and modify it so you extract well column and well row separately
regex = r"(?P<basepath>.*)[/\\].*images_(?P<Plate>.*)w\d[/\\](?P<Prefix>.*)_(?P<well>[A-Z]\d\d)_s(?P<subpos>\d)_w(?P<Channel>\d)(?P<ID>.*)\.tif$"

In [ ]:
firstimage = str(first_5_files[0])

In [ ]:
m = re.match(regex, firstimage)

In [ ]:
tmp = m.groupdict()
tmp

In [ ]:
tmp["filename"] = firstimage
tmp

In [ ]:
pd.Series(tmp) # convert the dictionary to a pd.Series

Let's assemble the individual steps we performed above into a single function.
Given a filename and a regular expression the function should extract metadata using the regular expression
and extract it. 

In [ ]:
def get_metadata_as_series(filepath, regex, filename_key="filepath"):
    ''' 
    provided with a filepath (can be a string or a pathlib.PosixPath object),
    tries to match the path against the regular expression regex.
    The extracted keys, plus the filepath are returned as a pandas Series object
    '''
    if  type(filepath) == pathlib.PosixPath:
        filepath = str(filepath)
    m = re.match(regex, filepath)
    if m is not None:
        tmp = m.groupdict()
        tmp[filename_key] = filepath
        return pd.Series(tmp)
    else:
        print(f"Extracting metadata for {filepath} failed.")
        return None

You can combine a list of multiple `pd.Series` objects into a `pd.DataFrame`. More on `DataFrames` later.

Let's try with two series:

In [ ]:
s1 = get_metadata_as_series(first_5_files[1], regex)
s2 = get_metadata_as_series(first_5_files[2], regex)

In [ ]:
pd.DataFrame([s1, s2])

# Find all files and extract metadata

Now let's create a data frame by analyzing all the filenames. 
There are several ways to do this: 
* You could use a for loop 
* You can use a `list comprehension`
* You can use `map`


In [ ]:
# create a fresh generator, because we've "used up" the previous one
files =  folder.rglob("*.tif") 
# for each file, extract the metadata ... using a list comprehension
metadata_series_list = [get_metadata_as_series(f, regex) for f in files]

Now combine all metadata series objects into a **pandas** `DataFrame`

In [ ]:
# there are many ways to create a DataFrame. Here we pass a list of pd.Series objects
df_meta = pd.DataFrame(metadata_series_list)

Just evaluating a pandas DataFrame in a cell will provide a tabular output (abbreviated for long data frames)

In [ ]:
df_meta

You can get some summary information for the data frame using `.describe()`

In [ ]:
df_meta.describe()

If you just want to get a quick feel for what kind of data is in a data frame, but you don't want to output a long frame use `.head()`

In [ ]:
df_meta.head()

### Interactive, sortable display of data frames using qgrid

Notice how we only get a static, abbreviated output of the large data frame in the notebook.
You can also display the dataframe in an interactive fashion using the qgrid package. To install, run
```
conda install -c conda-forge qgrid 
```
Documentation and examples can be found at https://github.com/quantopian/qgrid

In [ ]:
import qgrid
tablewidget = qgrid.show_grid(df_meta, show_toolbar=True)
tablewidget

You can actually edit the data frame with the qgrid widget and read back the contents into a data frame (see the documentatoin for details). 

This may be convenient, but from the perspective of reproducible data analysis such manual changes are not recommended unless you save your modified data frame, such that your changes can be reproduced from a file.


now we have already used `pd.Series` and `pd.DataFrame`, but ...

# ... what is Pandas ?
**Pandas** is a library that provides a `DataFrame` data structure and functions to work efficiently on this data structure. This is equivalent to a `DataFrame` or `tibble` in R. 
If you are not familiar with the concept of data frames, just think of it as an Excel table or a table in a relational database.

## How does a Pandas DataFrame differ from a numpy array ?
* `numpy` arrays are homogeneous, in the sense that they contain the same data type for each member of the array.
* `numpy` arrays support `n`-dimensional data, where `n` can be larger than 2.
* `pandas.DataFrame` objects can be heterogeneous, i.e. every column can hold a different object type. This includes non-numerical data.
* `pandas.DataFrame` objects are essentially 2-dimensional


## What do Pandas and numpy have in common ?

* they have similar interfaces for working with the data. For example, you can perform operation such as `some_array.mean()` or `some_array.unique()` etc.
* under the hood, a pandas dataframe uses numpy arrays to store the data


## Why bother?

You won't be storing images in pandas data frames, so why bother?

Typically, the aim of your image analysis is to go from images to measurements, e.g. a list of objects and measured features for these objects. Pandas data frames are ideal data structures for storing and analysing such lists. 
Pandas data frames support grouping operations (`groupby`) that let you efficiently run operations on certain subsets of the data, following the [Split-Apply-Combine Pattern](https://www.jstatsoft.org/article/view/v040i01/v40i01.pdf)
popularized by _Hadley Wickham_.
For instance if you have a data table that holds measurements taken in multiple plates of a high-throughput-screen you can quickly calculate statistics on a per-plate level if you group by plate. [You can find more examples in the pandas documentation](http://pandas.pydata.org/pandas-docs/stable/groupby.html).

# Learning Pandas
There is not enough time to cover pandas in depth during this course. For an introduction from the ground up,
check out Jake VanDerPlas's [Python Data Science Handbook](https://github.com/jakevdp/PythonDataScienceHandbook).
[Direct link to the pandas chapter](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.00-Introduction-to-Pandas.ipynb).

Alternatively there are also notebooks available for Wes McKinney's book [Python for Data Analysis](https://github.com/wesm/pydata-book). 
[Chapter 5 introduces Pandas](http://nbviewer.jupyter.org/github/pydata/pydata-book/blob/2nd-edition/ch05.ipynb)

In [ ]:
df_meta.head()

In [ ]:
# Let's look which wells we have images from:
df_meta["well"].unique()

In [ ]:
# note: different way of referring to the column, an alternative to df_meta["subpos"]
df_meta.subpos.unique()

Exporting/Importing a data frame

* `.csv` files
* `.json` files
* `pickle`, `hdf5`, `sql`

and system clipboard. 
You can try 
```
df_meta.to_clipboard()
```

and using paste in Excel. 
Or try copying something in Excel and running

```
pd.read_clipboard()
```

In [ ]:
# We don't really need the columns ID, Prefix and basepath for our further analysis, so let's get rid of them
df_meta.drop(columns=['ID', 'Prefix', 'basepath'], inplace=True)

In [ ]:
df_meta # or use qgrid.show_grid(df_meta) for a more fancy output

# DataFrame `groupby` method 

Our data frame has one row for each image file. However, some of these images clearly belong together, that is the images of the different fluorescence channels taken in the same _subposition_ of the same _well_ on the same _plate_.
Therefore we want to group these images together. This can be done naturally using the `groupby()` method of the pandas DataFrame. 

In [ ]:
groupby = df_meta.groupby(["well", "subpos", "Plate"])
groupby

In [ ]:
# one can iterate over groups
for group in groupby:
    print(group)

In [ ]:
# one can get the group keys
keys = groupby.groups.keys()
# show only the first few
list(keys)[:10]

In [ ]:
# access an individual group by key
groupby.get_group(list(keys)[0])

In [ ]:
# pick a group as an example, here is randomly chose the group with index 8
example_group = groupby.get_group(list(keys)[8])
example_group

In [ ]:
# these happen to be sorted, already but to make sure we can do                                  
example_group.sort_values("Channel", ascending=True) # also try ascending=False to see that it is actually sorting                                  
                                  

In [ ]:
# now just grab the filepaths
example_group.sort_values("Channel", ascending=True)["filepath"]

In [ ]:
# but this is still a series, lets turn it into a list
filelist = list(example_group.sort_values("Channel", ascending=True)["filepath"])
filelist

In [ ]:
# now we can read all of these images very quickly using a list comprehension or a map
images = map(imread, filelist)

In [ ]:
images

Images is a `map` object. Similarly to generators the mapping is only evaluated when it is needed. 
We can force the evaluation (in this case reading the images) by converting it to a list.

In [ ]:
images = list(images)
images

Let's convert the list of numpy arrays into an _n_-dimensional array (n=3)

In [ ]:
all_in_one = np.array(images)
all_in_one.shape

## Interactive plotting and image viewing using holoviews/bokeh

**Bokeh** is a plotting library similar to `matplotlib`. However, instead of generating static plots it generates plots as HTML-files with Javascript that can be embedded in the notebook and allow user interaction.

**Holoviews** is a higher-level plotting library that can use **bokeh** and **matplotlib** as backened. 

In [ ]:
import holoviews as hv
hv.extension('bokeh')  # without speciyfing the extension you won't see a plot

In [ ]:
imview = hv.Image(all_in_one[3,:,:]).options(tools=['hover'], cmap="gray",width=696, height=520, colorbar=False)
imview

With holoviews you can create an image viewer with a channel slider using their  `DynamicMap`.

In [ ]:
def select_channel(c):
    tmp = all_in_one[c,: , :]
    size = tmp.shape
    return  hv.Image(tmp).options(tools=['hover'], cmap="gray", width=size[1], height=size[0])

hv.DynamicMap(select_channel, kdims=['c',]).redim.values(c=range(5))

Now wrap this in a convenience function, so you can just run this on any image (note that in the cell above the callback  function referred directly to `all_in_one`)

In [ ]:
def viewer_with_channel(image_ch):
    def select_ch(c):
        tmp = image_ch[c,: , :]
        size = tmp.shape
        return  hv.Image(tmp).options(tools=['hover'], cmap="gray", width=size[1], height=size[0])
    
    return(hv.DynamicMap(select_ch, kdims=['c',]).redim.values(c=range(image_ch.shape[0])))

In [ ]:
viewer_with_channel(all_in_one)

If you are keen, you can also try to add additional sliders for adjusting the range, the colormap etc.
Here is a rough cut piece of code that demonstrates this functionality:
https://github.com/VolkerH/my_hv_gallery/blob/master/Images_with_interactors/Dynamic_map_interactor.ipynb

# Build a simple image analysis pipeline

* missing: Preprocessing (noise removal, illumination correction, background subraction)
* Segment nuclei using OTSU
* Split and label nuclei
* Expand to find cytoplasm
* missing: remove touching objects


In [ ]:
import skimage.filters # this module provides the otsu algorithm

Segment Nuclei using thresholding.
Determine the threshold value using Otsu's method of maximizing the inter-class variance. 
(https://en.wikipedia.org/wiki/Otsu's_method)

In [ ]:
# TODO ... put the nuclear channel in im
threshval = skimage.filters.threshold_otsu(im)
threshval

In [ ]:
hv.Image(im > threshval).options(cmap="gray",width=500, colorbar=True, tools=['hover'])

# Object splitting and connected component labelling

The following function takes a binary image and tries to split adjacent nuclei using the distance transform and finding local maxima.

In [ ]:
def split_and_label(thresholded_image, bc_size = (9,9)):
    
    '''split objects using distance transform and watershed
    this implementation uses functions from the mahotas package
    
    You could also try to implement this using scikit-image and scipy.ndimage functions
    such as scipy.ndimage.morphology.distance_transform_edt for the distance transform
    and peak_local_max to find the regional maxima of the seed points
    see for example here: scipy.ndimage.morphology.distance_transform_edt
    ''' 
    distances = mahotas.stretch(mahotas.distance(thresholded_image)) # you could try using 
    Bc = np.ones(bc_size) 
    maxima = mahotas.morph.regmax(distances, Bc=Bc) # you could try adapting this to s
    spots, n_spots = mahotas.label(maxima) #, Bc=Bc)
    surface = (distances.max() - distances)
    areas = mahotas.cwatershed(surface, spots)
    areas *= thresholded_image
    return(areas)

In [ ]:
# todo 

# try segmenting the nuclei using otsu and split and label




# Finding the cytoplasm

Once you have the nuclei as seed points, you can use several methods to grow these seed regions to find the surrounding cytoplasm. There are a number of commonly used techniques, for example, CellProfiler provides the following techniques:

* watershed
* seeded region growing
* distance-N

Unless you have a marker that clearly delineates the cell boundary or marks the whole cytoplasm, you should use distance-N, otherwise you might bias your results (interactive whiteboard: explain why).

In [ ]:
def distanceN(labels_in, distance):
    '''
    Distance-N implementation 
    Taken/adapted from the CellProfiler source code for their IdentifySecondaryObjects
    module.
    
    The basic idea is that you have some seed labels (in the context 
    of cell profiling these will typically be cell nuclei) that you want 
    to grow by n pixels to give a mask for a larger object (the cytoplasm).
    
    If you were only dealing with a single seed object, you could simply dilate with 
    a suitably sized structuring element. However, in general you have multiple seed 
    points and you don't want to merge those. Distance N will grow up to N pixels without
    merging objects that are closer together than 2N. 
    ''' 
    
    tmp = scipy.ndimage.morphology.distance_transform_edt(labels_in == 0, return_indices = True)
    distances, (i,j) = tmp
    labels_out = np.zeros(labels_in.shape, int)
    dilate_mask = distances <= distance
    labels_out[dilate_mask] = labels_in[i[dilate_mask],j[dilate_mask]]
    return labels_out    

# Feature extraction for the cell regions in each channel

**TODO:**

* Read the documentation of [`skimage.measure`](http://scikit-image.org/docs/dev/api/skimage.measure.html), in particular `regionprops`.
* Apply `regionprops` using a label image and a greyvalue image
* Try and make sense of the output
* assemble into a data frame
* save a crop or thumbnail for each segmented cell